<div>
    <div style="float:left;">
        <img src="http://oproject.org/tiki-download_file.php?fileId=8&display&x=450&y=128" width="50%" />
    </div>
    <div style="float:left;">
        <img src="http://gfif.udea.edu.co/root/tmva/img/tmva_logo.gif" width="50%"/>
    </div>
</div>

# Evaluate methods 
<hr style="border-top-width: 4px; border-top-color: #34609b;">

<!--<script src="JsRoot/scripts/JSRootCore.js?jq2d&onload=JsRootLoadedCall" type="text/javascript"></script>-->

In [1]:
import ROOT
from ROOT import TFile, TMVA, TCut

Welcome to JupyROOT 6.07/07


## Enable JS visualization

In [2]:
%jsmva on

## Declarations, building training and testing trees 

<b>For more details please see <a href="ROOTbooks-TMVA-JsMVA-UserInterface.ipynb">this</a> notebook.</b>

In [3]:
outputFile = TFile( "TMVA.root", 'RECREATE' )

TMVA.Tools.Instance()

factory = TMVA.Factory(JobName="TMVAClassification", TargetFile=outputFile,
            V=False, Color=True, DrawProgressBar=True, Transformations=["I", "D", "P", "G","D"],
                       AnalysisType="Classification")

dataset = "tmva_class_example"
loader  = TMVA.DataLoader(dataset)

loader.AddVariable( "myvar1 := var1+var2", 'F' )
loader.AddVariable( "myvar2 := var1-var2", "Expression 2", 'F' )
loader.AddVariable( "var3",                "Variable 3", 'F' )
loader.AddVariable( "var4",                "Variable 4", 'F' )

loader.AddSpectator( "spec1:=var1*2",  "Spectator 1",  'F' )
loader.AddSpectator( "spec2:=var1*3",  "Spectator 2",  'F' )

if ROOT.gSystem.AccessPathName( "./tmva_class_example.root" ) != 0: 
    ROOT.gSystem.Exec( "wget https://root.cern.ch/files/tmva_class_example.root")
    
input = TFile.Open( "./tmva_class_example.root" )

# Get the signal and background trees for training
signal      = input.Get( "TreeS" )
background  = input.Get( "TreeB" )
    
# Global event weights (see below for setting event-wise weights)
signalWeight     = 1.0
backgroundWeight = 1.0

mycuts = TCut("")
mycutb = TCut("")

loader.AddSignalTree(signal, signalWeight)
loader.AddBackgroundTree(background, backgroundWeight)
loader.fSignalWeight = signalWeight
loader.fBackgroundWeight = backgroundWeight
loader.fTreeS = signal
loader.fTreeB = background

loader.PrepareTrainingAndTestTree(SigCut=mycuts, BkgCut=mycutb,
            nTrain_Signal=0, nTrain_Background=0, SplitMode="Random", NormMode="NumEvents", V=False)

--- Factory                  : You are running ROOT Version: 6.07/07, Apr 1, 2016
--- Factory                  : 
--- Factory                  : _/_/_/_/_/ _|      _|  _|      _|    _|_|   
--- Factory                  :    _/      _|_|  _|_|  _|      _|  _|    _| 
--- Factory                  :   _/       _|  _|  _|  _|      _|  _|_|_|_| 
--- Factory                  :  _/        _|      _|    _|  _|    _|    _| 
--- Factory                  : _/         _|      _|      _|      _|    _| 
--- Factory                  : 
--- Factory                  : ___________TMVA Version 4.2.1, Feb 5, 2015
--- Factory                  : 
--- DataSetInfo              : Dataset[tmva_class_example] : Added class "Signal"	 with internal class number 0
--- tmva_class_example       : Add Tree TreeS of type Signal with 6000 events
--- DataSetInfo              : Dataset[tmva_class_example] : Added class "Background"	 with internal class number 1
--- tmva_class_example       : Add Tree TreeB of type Backgrou

## Booking methods

For more details please see <a href="ROOTbooks-TMVA-Training.ipynb"> this</a> notebook.

In [4]:
factory.BookMethod( DataLoader=loader, Method=TMVA.Types.kSVM, MethodTitle="SVM", 
                    Gamma=0.25, Tol=0.001, VarTransform="Norm" )

factory.BookMethod( DataLoader=loader, Method=TMVA.Types.kMLP, MethodTitle="MLP", 
                    H=False, V=False, NeuronType="tanh", VarTransform="N", NCycles=600, HiddenLayers="N+5",
                    TestRate=5, UseRegulator=False )

factory.BookMethod( DataLoader=loader, Method=TMVA.Types.kLD, MethodTitle="LD", 
                    H=False, V=False, VarTransform="None", CreateMVAPdfs=True, PDFInterpolMVAPdf="Spline2",
                    NbinsMVAPdf=50, NsmoothMVAPdf=10 )

trainingStrategy = [{
        "LearningRate": 1e-1,
        "Momentum": 0.0,
        "Repetitions": 1,
        "ConvergenceSteps": 300,
        "BatchSize": 20,
        "TestRepetitions": 15,
        "WeightDecay": 0.001,
        "Regularization": "NONE",
        "DropConfig": "0.0+0.5+0.5+0.5",
        "DropRepetitions": 1,
        "Multithreading": True
        
    }, {
        "LearningRate": 1e-2,
        "Momentum": 0.5,
        "Repetitions": 1,
        "ConvergenceSteps": 300,
        "BatchSize": 30,
        "TestRepetitions": 7,
        "WeightDecay": 0.001,
        "Regularization": "L2",
        "DropConfig": "0.0+0.1+0.1+0.1",
        "DropRepetitions": 1,
        "Multithreading": True
        
    }, {
        "LearningRate": 1e-2,
        "Momentum": 0.3,
        "Repetitions": 1,
        "ConvergenceSteps": 300,
        "BatchSize": 40,
        "TestRepetitions": 7,
        "WeightDecay": 0.001,
        "Regularization": "L2",
        "Multithreading": True
        
    },{
        "LearningRate": 1e-3,
        "Momentum": 0.1,
        "Repetitions": 1,
        "ConvergenceSteps": 200,
        "BatchSize": 70,
        "TestRepetitions": 7,
        "WeightDecay": 0.001,
        "Regularization": "NONE",
        "Multithreading": True
        
}]

factory.BookMethod(DataLoader=loader, Method=TMVA.Types.kDNN, MethodTitle="DNN", 
                   H = False, V=False, VarTransform="Normalize", ErrorStrategy="CROSSENTROPY",
                   Layout=["TANH|100", "TANH|50", "TANH|10", "LINEAR"],
                   TrainingStrategy=trainingStrategy)

factory.BookMethod(loader, TMVA.Types.kLikelihood, "Likelihood", 
                   "NSmoothSig[0]=20:NSmoothBkg[0]=20:NSmoothBkg[1]=10:NSmooth=1:NAvEvtPerBin=50",
                    H=True, V=False,TransformOutput=True,PDFInterpol="Spline2")

factory.BookMethod(DataLoader= loader, Method=TMVA.Types.kBDT, MethodTitle="BDT",
                   H=False,V=False,NTrees=850,MinNodeSize="2.5%",MaxDepth=3,BoostType="AdaBoost", AdaBoostBeta=0.5,
                   UseBaggedBoost=True,BaggedSampleFraction=0.5, SeparationType="GiniIndex", nCuts=20 )

<ROOT.TMVA::MethodBDT object ("BDT") at 0x61fb940>

--- Factory                  : Booking method: SVM DataSet Name: tmva_class_example
--- SVM                      : Dataset[tmva_class_example] : Create Transformation "Norm" with events from all classes.
--- Norm                     : Transformation, Variable selection : 
--- Norm                     : Input : variable 'myvar1' (index=0).   <---> Output : variable 'myvar1' (index=0).
--- Norm                     : Input : variable 'myvar2' (index=1).   <---> Output : variable 'myvar2' (index=1).
--- Norm                     : Input : variable 'var3' (index=2).   <---> Output : variable 'var3' (index=2).
--- Norm                     : Input : variable 'var4' (index=3).   <---> Output : variable 'var4' (index=3).
--- Factory                  : Booking method: MLP DataSet Name: tmva_class_example
--- MLP                      : Dataset[tmva_class_example] : Create Transformation "N" with events from all classes.
--- Norm                     : Transformation, Variable selection : 
--- Norm 

# Train Methods

In [5]:
factory.TrainAllMethods()

--- Norm                     : Preparing the transformation.
--- TFHandler_SVM            : -----------------------------------------------------------
--- TFHandler_SVM            : Variable        Mean        RMS   [        Min        Max ]
--- TFHandler_SVM            : -----------------------------------------------------------
--- TFHandler_SVM            :   myvar1:   0.083989    0.36407   [    -1.0000     1.0000 ]
--- TFHandler_SVM            :   myvar2:  0.0094778    0.27696   [    -1.0000     1.0000 ]
--- TFHandler_SVM            :     var3:   0.080279    0.36720   [    -1.0000     1.0000 ]
--- TFHandler_SVM            :     var4:    0.12986    0.39603   [    -1.0000     1.0000 ]
--- TFHandler_SVM            : -----------------------------------------------------------
--- SVM                      : Dataset[tmva_class_example] : Begin training
--- SVM                      : Building SVM Working Set...with 6000 event instances
--- SVM                      : Elapsed time for Wor

## Test end evaluate the methods

To test test the methods and evaluate the performance we need to run Factory.TestAllMethods and Factory.EvaluateAllMethods functions.

In [6]:
factory.TestAllMethods()
factory.EvaluateAllMethods()

--- Factory                  : Test all methods...
--- Factory                  : Test method: SVM for Classification performance
--- SVM                      : Dataset[tmva_class_example] : Evaluation of SVM on testing sample (6000 events)
--- SVM                      : Dataset[tmva_class_example] : Elapsed time for evaluation of 6000 events: 0.997 sec       
--- Factory                  : Test method: MLP for Classification performance
--- MLP                      : Dataset[tmva_class_example] : Evaluation of MLP on testing sample (6000 events)
--- MLP                      : Dataset[tmva_class_example] : Elapsed time for evaluation of 6000 events: 0.0116 sec       
--- Factory                  : Test method: LD for Classification performance
--- LD                       : Dataset[tmva_class_example] : Evaluation of LD on testing sample (6000 events)
--- LD                       : Dataset[tmva_class_example] : Elapsed time for evaluation of 6000 events: 0.00415 sec       
--- LD      

## Classifier Output Distributions

To draw the classifier output distribution we have to use Factory.DrawOutputDistribution function which is inserted by invoking jsmva magic. The parameters of the function are the following:
The options string can contain the following options:
<table>
<tr><th>Keyword</th><th>Can be used as positional argument</th><th>Default</th><th>Predefined values</th><th>Description</th></tr>
<tr>
    <td>datasetName</td>
    <td>yes, 1.</td>
    <td>-</td>
    <td>-</td>
    <td> The name of dataset</td>
</tr>
<tr>
    <td>methodName</td>
    <td>yes, 2.</td>
    <td>-</td>
    <td>-</td>
    <td> The name of method</td>
</tr>
</table>

In [7]:
factory.DrawOutputDistribution(dataset, "MLP")

## Classifier Probability Distributions

To draw the classifier probability distribution we have to use Factory.DrawProbabilityDistribution function which is inserted by invoking jsmva magic. The parameters of the function are the following:
The options string can contain the following options:
<table>
<tr><th>Keyword</th><th>Can be used as positional argument</th><th>Default</th><th>Predefined values</th><th>Description</th></tr>
<tr>
    <td>datasetName</td>
    <td>yes, 1.</td>
    <td>-</td>
    <td>-</td>
    <td> The name of dataset</td>
</tr>
</table>

In [8]:
factory.DrawProbabilityDistribution(dataset, "LD")

## ROC curve

To draw the ROC (receiver operating characteristic) curve we have to use Factory.DrawROCCurve function which is inserted by invoking jsmva magic. The parameters of the function are the following:
The options string can contain the following options:
<table>
<tr><th>Keyword</th><th>Can be used as positional argument</th><th>Default</th><th>Predefined values</th><th>Description</th></tr>
<tr>
    <td>datasetName</td>
    <td>yes, 1.</td>
    <td>-</td>
    <td>-</td>
    <td> The name of dataset</td>
</tr>
</table>

In [9]:
factory.DrawROCCurve(dataset)

## Classifier Cut Efficiencies

To draw the classifier cut efficiencies we have to use Factory.DrawCutEfficiencies function which is inserted by invoking jsmva magic. The parameters of the function are the following:
The options string can contain the following options:
<table>
<tr><th>Keyword</th><th>Can be used as positional argument</th><th>Default</th><th>Predefined values</th><th>Description</th></tr>
<tr>
    <td>datasetName</td>
    <td>yes, 1.</td>
    <td>-</td>
    <td>-</td>
    <td> The name of dataset</td>
</tr>
<tr>
    <td>methodName</td>
    <td>yes, 2.</td>
    <td>-</td>
    <td>-</td>
    <td> The name of method</td>
</tr>
</table>

In [10]:
factory.DrawCutEfficiencies(dataset, "MLP")

## Close the factory's output file

In [11]:
outputFile.Close()